# Горбунов Иван

## HW2 Вариант 2:

оцените дистанцию поездок (в метрах) на основе координат начальной и конечной станций (без учета поездок, завершившихся там же где и начались)
выведите максимальное, среднее значение, стандартное отклонение и медиан


In [18]:
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

#Настройки для Mac
os.environ["OBJC_DISABLE_INITIALIZE_FORK_SAFETY"] = "YES"
os.environ["PYSPARK_PYTHON"] = "python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/temurin-17.jdk/Contents/Home"

# Запуск Spark Session
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("HW2_Task2_CitiBike") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print("Spark DataFrame API запущен")

# Чтение файла
csv_name = "JC-201902-citibike-tripdata.csv"

print(f"Чтение файла {csv_name}...")
df = spark.read.csv(csv_name, header=True, inferSchema=True)
# Загрузка данных
print(f"Загрука данных из {csv_name}...")
df = spark.read.csv(csv_name, header=True, inferSchema=True)

# Выполнение Задания 2

# Радиус Земли (метры)
R = 6371000 

# Фильтр: исключаем поездки, где начало и конец совпадают (по условию)
filtered_df = df.filter(col("start station id") != col("end station id"))

# Переводим координаты в радианы для формулы
lat1 = F.radians(col("start station latitude"))
lon1 = F.radians(col("start station longitude"))
lat2 = F.radians(col("end station latitude"))
lon2 = F.radians(col("end station longitude"))

# Формула гаверсинуса (расстояние по сфере)
dlon = lon2 - lon1
dlat = lat2 - lat1
a = F.pow(F.sin(dlat / 2), 2) + F.cos(lat1) * F.cos(lat2) * F.pow(F.sin(dlon / 2), 2)
c = 2 * F.atan2(F.sqrt(a), F.sqrt(1 - a))

# Добавляем колонку с дистанцией
df_dist = filtered_df.withColumn("distance_meters", R * c)

print("Дистанции рассчитаны. Подсчет статистики...")

# Считаем Макс, Среднее, Стд. отклонение, Медиану
stats = df_dist.select(
    F.max("distance_meters").alias("Максимум (м)"),
    F.mean("distance_meters").alias("Среднее (м)"),
    F.stddev("distance_meters").alias("Стандартное откл."),
    F.percentile_approx("distance_meters", 0.5).alias("Медиана (м)")
)

print("\n" + "="*40)
print("РЕЗУЛЬТАТ")
stats.show(truncate=False)

Spark DataFrame API запущен
Чтение файла JC-201902-citibike-tripdata.csv...
Загрука данных из JC-201902-citibike-tripdata.csv...
Дистанции рассчитаны. Подсчет статистики...

РЕЗУЛЬТАТ
+-----------------+-----------------+-----------------+-----------------+
|Максимум (м)     |Среднее (м)      |Стандартное откл.|Медиана (м)      |
+-----------------+-----------------+-----------------+-----------------+
|4975.170135370611|963.2505212560991|570.8997349563779|829.7748149644184|
+-----------------+-----------------+-----------------+-----------------+



26/01/11 05:45:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1046924 ms exceeds timeout 120000 ms
26/01/11 05:45:42 WARN SparkContext: Killing executors is not supported by current scheduler.
26/01/11 05:45:50 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:359)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:674)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1363)
	at 